非关系型数据库主要有下面这几种

![](.3_images/578f0ae6.png)

## monoDB

In [19]:
import pymongo

# 首先连接一下数据库
client = pymongo.MongoClient(host='192.168.1.50', port=30001)
# 列出所有数据库名字
print(client.list_database_names())
# 然后指定一下数据库
# db = client.demo
# 下面这样也可以
db = client['demo']

['aaa', 'admin', 'config', 'demo', 'local']


In [20]:
# 再指定一下集合
# collection  = db.students
# 下面这样也可以
collection = db['student']

In [28]:
# 指定好集合后就可以开始准备插入数据了
student = {
    'id': '20170101',
    'name': 'Jordan',
    'age': 20,
    'gender': 'male'
}
# 插入我们的数据(不要用insert方法，那个方法已经没用了)
result = collection.insert_one(student)
print(result)

In [24]:
# 我们还可以插入多条数据
student1 = {
    'id': '20170101',
    'name': 'Jordan',
    'age': 20,
    'gender': 'male'
}

student2 = {
    'id': '20170202',
    'name': 'Mike',
    'age': 21,
    'gender': 'male'
}
# 多条数据需要使用insert_many方法
result = collection.insert_many([student1, student2])
print(result)

In [25]:
# 下面演示一下查询操作，我们查询一下name为mike的数据
result = collection.find_one({'name': 'Mike'})
print(type(result))
print(result)

<class 'dict'>
{'_id': ObjectId('62a13b9df33902758f88e1a8'), 'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}


In [27]:
# 也可以使用objectid来进行查询，这里需要引入对应的对象，这个id要换成自己的
from bson.objectid import ObjectId

result = collection.find_one({'_id': ObjectId('62a13b9df33902758f88e1a7')})
print(result)

{'_id': ObjectId('62a13b9df33902758f88e1a7'), 'id': '20170101', 'name': 'Jordan', 'age': 20, 'gender': 'male'}


In [29]:
# 下面演示一下多条数据查询
# 这个返回的是一个游标对象
results = collection.find({'age': 20})
print(results)
# 这里直接使用for循环遍历即可
for result in results:
    print(result)

{'_id': ObjectId('62a13b9df33902758f88e1a7'), 'id': '20170101', 'name': 'Jordan', 'age': 20, 'gender': 'male'}
{'_id': ObjectId('62a13c93f33902758f88e1a9'), 'id': '20170101', 'name': 'Jordan', 'age': 20, 'gender': 'male'}


In [30]:
# 我们可以通过下面这个来查询年龄大于20的对象
results = collection.find({'age': {'$gt':20}})
for result in results:
    print(result)

{'_id': ObjectId('62a13b9df33902758f88e1a8'), 'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}


![](.3_images/e81c311b.png)

In [31]:
# 也可以直接使用正则匹配
results = collection.find({'name': {'$regex': '^M.*'}})
for result in results:
    print(result)

{'_id': ObjectId('62a13b9df33902758f88e1a8'), 'id': '20170202', 'name': 'Mike', 'age': 21, 'gender': 'male'}


![](.3_images/275f6a79.png)

In [32]:
# 如果想知道查询结果有多少条数据，可以使用计数功能（这个功能在最新版本已经失效了）
count = collection.find().count()
print(count)

AttributeError: 'Cursor' object has no attribute 'count'

In [33]:
# 我们可以对查询到的结果进行排序（这里是升序排序）
results = collection.find().sort('name', pymongo.ASCENDING)
print([result['name'] for result in results])

['Jordan', 'Jordan', 'Mike']


In [34]:
# 我们可以使用skip来偏移位置（比如这里我们跳过前两个数据）
results = collection.find().sort('name', pymongo.ASCENDING).skip(2)
print([result['name'] for result in results])

['Mike']


In [35]:
# 我们这里可以限制返回的数据条数
results = collection.find().sort('name', pymongo.ASCENDING).limit(2)
print([result['name'] for result in results])

['Jordan', 'Jordan']


In [45]:
# 下面演示一下更新数据
# 首先查找一下数据
condition = {'name': 'Mike'}
student = collection.find_one(condition)
print(student)
# 然后我们修改一下数据
student['age'] = 25
# 最后更新一下这个数据，这里需要使用update one，然后使用set来设置对象
result = collection.update_one(condition, {'$set':student})
print(result)
print(result.matched_count, result.modified_count)

{'_id': ObjectId('62a13b9df33902758f88e1a8'), 'id': '20170202', 'name': 'Mike', 'age': 25, 'gender': 'male'}
1 0


In [46]:
# 指定查询条件为年龄大于20，然后直接修改年龄并+1，然后之前为25的，现在改成了26
condition = {'age': {'$gt': 20}}
# 如果使用update_many的话就会全部更新了
result = collection.update_one(condition, {'$inc': {'age': 1}})
print(result)
print(result.matched_count, result.modified_count)

1 1


In [48]:
# 下面是删除操作
# delete_one删除一条，delete_many删除多条
result = collection.delete_one({'name': 'Kevin'})
print(result)

## redis

In [50]:
from redis import StrictRedis
# 建立连接
redis = StrictRedis(host='192.168.1.50', port=30319, db=0)
# 直接插入一条数据
redis.set('name', 'Bob')
print(redis.get('name'))

b'Bob'


In [ ]:
from redis import StrictRedis, ConnectionPool
# 我们可以创建一个连接池
pool = ConnectionPool(host='192.168.1.50', port=30319, db=0)
redis = StrictRedis(connection_pool=pool)

![](.3_images/b2cacdbc.png)

![](.3_images/f0f8135a.png)

![](.3_images/92bf8c21.png)

![](.3_images/4cc22107.png)

![](.3_images/4f23b983.png)

![](.3_images/a03b76bf.png)

![](.3_images/77198242.png)

![](.3_images/210e4c65.png)

![](.3_images/1878483f.png)

![](.3_images/bc099428.png)

![](.3_images/4430adbd.png)

![](.3_images/c662cf5a.png)